In [7]:
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
print(X.shape, y.shape)
n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1234)

sc = StandardScaler() # make our dataset have 0 mean and unit variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)
print(y_train.shape, y_test.shape)

(569, 30) (569,)
torch.Size([455, 1]) torch.Size([114, 1])


In [11]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, in_features):
        super().__init__()

        self.layer1 = nn.Linear(in_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.layer1(x))
    

model = LogisticRegressionModel(n_features)

In [12]:
model

LogisticRegressionModel(
  (layer1): Linear(in_features=30, out_features=1, bias=True)
)

In [15]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    y_hat = model(X_train)
    loss = criterion(y_hat, y_train)

    loss.backward() # compute gradient of this tensor wrt graph leaves

    optimizer.step() # update the weights

    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        # run validation every 10 epochs
        with torch.no_grad():
            y_hat = model(X_test)
            y_hat_classes = y_hat.round()
            acc = y_hat_classes.eq(y_test).sum() / float(y_test.shape[0])
            print(f'epoch: {epoch + 1} loss: {loss.item():.4f} accuracy: {acc:.4f}')


epoch: 10 loss: 0.1721 accuracy: 0.9035
epoch: 20 loss: 0.1687 accuracy: 0.9035
epoch: 30 loss: 0.1656 accuracy: 0.9035
epoch: 40 loss: 0.1626 accuracy: 0.9035
epoch: 50 loss: 0.1598 accuracy: 0.9035
epoch: 60 loss: 0.1572 accuracy: 0.9035
epoch: 70 loss: 0.1547 accuracy: 0.9123
epoch: 80 loss: 0.1523 accuracy: 0.9123
epoch: 90 loss: 0.1500 accuracy: 0.9123
epoch: 100 loss: 0.1479 accuracy: 0.9123
